<a href="https://colab.research.google.com/github/pyDesi/databrief/blob/main/Reference_data_brief.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Reference Notebook For Google Colab For MPH Data Brief Data Collection, being shared for Replicability, 8/17/2021

In [ ]:

!pip install censusdata
import pandas as pd
import censusdata


states = censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2019) 
print(states)
print(states['Pennsylvania']) #==>gives me the state code of 42
counties = censusdata.geographies(censusdata.censusgeo([('state', '42'), ('county', '*')]), 'acs5', 2019)
print(counties)

data = censusdata.download('acs5', 2019,
           censusdata.censusgeo([('state', '42'),
                                 ('county', '*')
                                 ]),
          ['B11012_001E','B01003_001E','B15003_017E','B15003_022E','C02003_003E','C02003_004E','C02003_005E','C02003_006E','C02003_007E','B28011_004E','B28008_004E','B19013_001E'])

column_names=['county_hh_count','total_population','high_school','bachelors','white','african american','American Indian/Alaskan Natives','Asian','Native Hawaiian/Pacific Islanders','computer in hh with broadband','per capita income 12 mo (2019adj$)','median hh income 12 mo (2019adj$)']
data.columns=column_names


In [ ]:
#Check and Make sure that the 67 Counties of the State of Pennsylvania are infact in the gleaned Dataset
counties.keys()
data['counties']=counties.keys()
data['counties'] #Check Lngth of column given at the end of the readout, should be 67

In [ ]:
#Computationally Derived Columns for the Dataset data
data['percent_households_with_bb']=(data['computer in hh with broadband'])/(data['county_hh_count'])*100
data['percent_pop_high_sch']=(data['high_school'])/(data['total_population'])*100
data['percent_pop_bachelors']=(data['bachelors'])/(data['total_population'])*100
data
#('C02003_003E', 'DETAILED RACE', 'Estimate!!Total:!!Population of one race:!!White')
#('C02003_004E', 'DETAILED RACE', 'Estimate!!Total:!!Population of one race:!!Black or African American')
#('C02003_005E', 'DETAILED RACE', 'Estimate!!Total:!!Population of one race:!!American Indian and Alaska Native')
#('C02003_006E', 'DETAILED RACE', 'Estimate!!Total:!!Population of one race:!!Asian')
#('C02003_007E', 'DETAILED RACE', 'Estimate!!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander')


In [ ]:
#Getting vaccination data for all counties of pennsylvania, obtained 13th of August 2021 from the State of Pennsylvania COVID-19 Dashboard
#which is located at COVID-19 Vaccine Dashboard at https://www.health.pa.gov/topics/disease/coronavirus/Pages/Cases.aspx from where data was downloaded
vax_data=pd.read_csv('/content/drive/MyDrive/data_brief_files/COVID-19_Vaccinations_by_Residence_Current_County_Health.csv')
vax_data['County Name']=vax_data['County Name']+' County, Pennsylvania'
vax_data.rename(columns={'County Name':'counties'},inplace=True)
vax_data

In [ ]:
#Now merging data and vax_data becaue I want to consolidate all data SED's and the outcomes (full/prtial and total vaccinations) per county in one file
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
merge_data=pd.merge(data,vax_data,how="inner",on='counties') #an inner join to merge vaccination data with the SED's
merge_data.loc[merge_data['counties']=='Somerset County, Pennsylvania']  #Just to check whether the merge went correctly by verifying manually

In [ ]:
#Just For manual confirmation Of the above merge
vax_data.loc[vax_data.counties == 'Somerset County, Pennsylvania']

In [ ]:
sorted_merge=merge_data.sort_values(by='County Population',ascending=False)  #Sorting merged data from above in descending order  
#some tweaks to see sorted_merge with all the columns and rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
sorted_merge
#To make for clearer data representation, I chopped it down to 40 rows in descending order
sorted_merge_top_40=sorted_merge.head(40)
sorted_merge_top_40


In [ ]:
data.to_csv('PA_county_Level_Edu_Race_Income_Broadband.csv') #initial SED
sorted_merge['fully vaccinated percentage']=sorted_merge['Fully Covered'] / sorted_merge['County Population'] *100
sorted_merge['partially vaccinated percentage']=sorted_merge['Partially Covered'] / sorted_merge['County Population'] *100
sorted_merge.to_csv('/content/drive/MyDrive/data_brief_files/merged_SED_vax.csv') #Merged SED Data with Vaccination Data

## END OF DATA COLLECTION... BELOW IS GRAPHING SCRATCH PAD ,PLEASE IGNORE

In [ ]:
sorted_merge

In [ ]:
new_indices=[]
county_names=[]
for index in data.index.to_list():
  new_index=index.geo[0][1] + index.geo[1][1]
  print(new_index)
  new_indices.append(new_index)
  county_name = index.name.split(',')[0]
  county_names.append(county_name)

data.index = new_indices
data['county_name'] = county_names

data



In [ ]:
data.index

In [ ]:
# import necessary libraries
!pip install geopandas
#!pip install shapefile
!pip install plotly-geo
!pip install pyshp 
import geopandas
import shapely
#import shapefile
import plotly

#import plotly-geo
from plotly.figure_factory._county_choropleth import create_choropleth
import xlrd
# Check your version
#print(plotly.__version__, geopandas.__version__,shapely.__version__,shapefile.__version__)


import geopandas
import shapely
#import shapefile
import plotly
#import plotly.figure_factory as ff
#fig = create_choropleth(fips=data.index, scope=['Pennsylvania'],values=data['african american'], title='PA Counties by High school education', legend_title='% high school')
fig=create_choropleth(fips=data.index.tolist(),scope=['Pennsylvania'],values=data['percent_households_with_bb'].tolist(),title='PA Counties by percentage Households with Broadband',legend_title='% households with broadband')
fig.layout.template = None
fig.show()


In [ ]:

fig=create_choropleth(fips=data.index.tolist(),scope=['Pennsylvania'],values=data['percent_households_with_bb'].tolist(),title='PA Counties by percentage Households with Broadband',legend_title='% households with broadband')
fig.layout.template = 'none'
fig.update_layout(legend = dict(font = dict(family = "Courier", size = 5, color = "black")))
fig.show()


In [ ]:
import plotly.express as px
  
df = px.data.iris()
  
fig = px.scatter(data, x="counties", y="percent_households_with_bb",
                 labels={"percent_households_with_bb": "percent households with broadband"},
                 size='percent_households_with_bb', color_continuous_scale=px.colors.sequential.algae_r)
  
fig.show()

In [ ]:
import plotly.express as px
import numpy as np
  
#df = px.data.iris()
  
#fig = px.scatter(data, x="counties", y="percent_households_with_bb",
                # color="counties", labels={"percent_households_with_bb": "<b>percent households with broadband</b>","counties":"<b>Pennsylvania Counties</b>"},
                # size='percent_households_with_bb',title="<b>Percentage Housholds with Broadband Access in Each County of Pennsylvania</b>",
                # width=2000, height=615,template = 'plotly_white')

sorted2_merge_top_40=sorted_merge_top_40.sort_values(by='County Population',ascending=True,inplace=True)


fig = px.line(sorted2_merge_top_40, x=sorted2_merge_top_40.Counties, y=sorted2_merge_top_40.percent_households_with_bb)


fig.update_layout(title_text='<b>2019 US Census based Percentage Housholds with Broadband Access in Each County of Pennsylvania</b>', title_x=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickfont_family="Arial Black")

#fig.update_xaxes(tickfont_family="Arial Black")


fig.show()